In [1]:
# now
proj_list = [
    'boringssl', 'c-ares',
    'freetype2', 'guetzli',
    'harfbuzz', 'lcms',
    'libarchive', 'libpng',
    'libssh', 'libxml2',
    'pcre2', 'proj4',
    're2', 'sqlite3',
    'vorbis', 'woff2',
    'wpantund'
]

# b4
proj_list = [
    'boringssl', 'c-ares',
    'freetype2', 'guetzli',
    'harfbuzz', 'lcms',
    'libpng', 'libssh',
    'libxml2', 'pcre2',
    'proj4', 're2',
    'sqlite3', 'vorbis',
    'woff2', 'wpantund'
]

In [2]:
target_project = 0

In [3]:
from sklearn.model_selection import train_test_split
import torch
import torch, gc

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import timeit

import data
import data_loader as dl
import initializer as init
import trainer
import tester
# import predictor
import model_util as mu

import os

In [4]:
gc.collect()
torch.cuda.empty_cache()

# print(torch.cuda.memory_summary(device=None, abbreviated=False))

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
# get all data exept target project
prefix_np, postfix_np, label_np, label_len_np = data.getTrainData(proj_list, proj_list[target_project])

Getting data for "boringssl" from "boringssl"
Getting data for "boringssl" from "c-ares"
Getting data for "boringssl" from "freetype2"
Getting data for "boringssl" from "guetzli"
Getting data for "boringssl" from "harfbuzz"
Getting data for "boringssl" from "lcms"
Getting data for "boringssl" from "libpng"
Getting data for "boringssl" from "libssh"
Getting data for "boringssl" from "libxml2"
Getting data for "boringssl" from "pcre2"
Getting data for "boringssl" from "proj4"
Getting data for "boringssl" from "re2"
Getting data for "boringssl" from "sqlite3"
Getting data for "boringssl" from "vorbis"
Getting data for "boringssl" from "woff2"
Getting data for "boringssl" from "wpantund"


In [6]:
# get target project data
test_prefix, test_postfix, test_label, test_label_len = data.getTestData(proj_list[target_project])

In [7]:
train_prefix, val_prefix, train_postfix, val_postfix, train_label, val_label = train_test_split(
    prefix_np, postfix_np, label_np, test_size = 0.2, random_state = 43
)

In [8]:
train_dataloader, val_dataloader, test_dataloader =\
    dl.data_loader(
        train_prefix, train_postfix,
        val_prefix, val_postfix,
        test_prefix, test_postfix,
        train_label, val_label, test_label,
        batch_size=1000
    )

In [9]:
overall_title = 'webModel2'

In [10]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/'+overall_title+'/tests')

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [12]:
# ====================
# set parameters here
# ====================

title = proj_list[target_project] + '_' + overall_title + '_3'
epochs = 40

max_len, source_code_tokens, token_choices = data.getInfo()

learning_rate = 0.001
weight_decay = 1e-4 

embed_dim = 100
hidden_size = 200
n_layers = 1
output_size = max(token_choices) + 1
dropout = 0.5
max_length = max_len
input_size = max(token_choices) + 1
device = device

model_name = "seq2seq"
optim_name = "Adam"
loss_fn_name = "CEL"

teacher_forcing_ratio = 100.0

In [13]:
trainer.set_seed(42)

loss_fn, prefix_pack, postfix_pack, attn_pack = init.initialize_model(
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    n_layers=n_layers,
    output_size=output_size,
    dropout=dropout,
    max_length=max_length,
    input_size=input_size,
    device=device
)

/home/arise/anaconda3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [14]:
start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    loss_fn=loss_fn,
    prefix_pack=prefix_pack,
    postfix_pack=postfix_pack,
    attn_pack=attn_pack,
    teacher_forcing_ratio=teacher_forcing_ratio
)

end_time = (timeit.default_timer() - start_time) / 60.0

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   6.300175   | 93.946492  | 9.668332 | 90.48  | 237.57
   2    |   5.623495   | 95.724905  | 9.549261 | 91.31  | 242.21
   3    |   5.518221   | 96.094107  | 9.235161 | 91.84  | 235.91
   4    |   5.459836   | 96.297321  | 9.025065 | 92.01  | 242.40
   5    |   5.419937   | 96.441176  | 8.893991 | 92.33  | 241.24
   6    |   5.390042   | 96.550546  | 8.797735 | 92.37  | 238.65
   7    |   5.365393   | 96.647311  | 8.720750 | 92.52  | 243.70
   8    |   5.344805   | 96.726071  | 8.761809 | 92.57  | 236.19
   9    |   5.326615   | 96.800273  | 8.627928 | 92.84  | 241.08
  10    |   5.310503   | 96.860042  | 8.619812 | 92.77  | 244.28
  11    |   5.294668   | 96.926691  | 8.531387 | 92.96  | 237.36
  12    |   5.281505   | 96.979485  | 8.482926 | 92.89  | 242.91
  13    |   5.269362   | 97.026870  | 8.536971 | 93.0

In [15]:
mu.saveModel(overall_title, title, prefix_pack, postfix_pack, attn_pack)

In [16]:
prefix_pack, postfix_pack, attn_pack = mu.getModel(overall_title, title)
# print(model)

In [17]:
loss, acc = tester.test(
    test_dataloader=test_dataloader,
    device=device,
    loss_fn=loss_fn,
    prefix_pack=prefix_pack,
    postfix_pack=postfix_pack,
    attn_pack=attn_pack
)

test loss:  7.628797934413072
test acc:  94.58054474708172


In [18]:
with open('../stat/'+overall_title, 'a') as f:
        text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\n'
        f.write(text)

In [19]:
# mu.graphModel(train_dataloader, model, writer)